# GBT

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pprint import pprint

In [18]:
# Load and parse the data file, converting it to a DataFrame
data = spark.read.format('libsvm').load('file:///Wang Hanmo/scripts/spark/sample_libsvm_data.txt')
pprint(data)


Py4JJavaError: An error occurred while calling o530.load.
: java.lang.UnsupportedOperationException: empty collection
	at org.apache.spark.rdd.RDD$$anonfun$reduce$1$$anonfun$apply$36.apply(RDD.scala:1031)
	at org.apache.spark.rdd.RDD$$anonfun$reduce$1$$anonfun$apply$36.apply(RDD.scala:1031)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD$$anonfun$reduce$1.apply(RDD.scala:1031)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.RDD.reduce(RDD.scala:1011)
	at org.apache.spark.mllib.util.MLUtils$.computeNumFeatures(MLUtils.scala:94)
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat$$anonfun$1.apply$mcI$sp(LibSVMRelation.scala:104)
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat$$anonfun$1.apply(LibSVMRelation.scala:95)
	at org.apache.spark.ml.source

In [7]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol='label', outputCol='indexedLabel').fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.


NameError: name 'data' is not defined